In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import skewnorm, norm

from src.utils import skew_to_alpha, moments
from src.sharpe_ratio_stats import estimated_sharpe_ratio, probabilistic_sharpe_ratio

## Strategy 1 - *Simulate a distribution of returns with negative skewness*

In [14]:
np.random.seed(0)

In [15]:
SIZE = 52
SR_BENCHMARK = 0

MEAN_RETURNS = 0.01
STD_RETURNS  = 0.01
SKEW_RETURNS = -0.99  # must be between [-0.99, 0.99]

alpha = skew_to_alpha(SKEW_RETURNS)

In [16]:
_seed = 1000
_sr  = 0.45
_psr = 0.96
_skew = SKEW_RETURNS

while ( (_psr < 0.92) or (_psr > 0.93) or (_skew < SKEW_RETURNS-0.3) or (_skew > SKEW_RETURNS+0.3) ) and _seed<5000:
    _seed += 1
    
    _dist_neg_skew = skewnorm(loc=MEAN_RETURNS, scale=STD_RETURNS, a=alpha)
    _returns = _dist_neg_skew.rvs(size=SIZE, random_state=_seed)
    
    _moments = moments(_returns)
    _skew = _moments.loc['skew']
    
    _sr  = estimated_sharpe_ratio(_returns)
    _psr = probabilistic_sharpe_ratio(_returns, sr_benchmark=SR_BENCHMARK)

    
# NOTE: scipy.skewnorm can not mantain the mean and std of the distribution while changing the skewness
true_mean = _dist_neg_skew.stats('m').item()
true_std  = np.sqrt(_dist_neg_skew.stats('v').item())

print(_seed)

1131


## Strategy 2 - *Simulate a normal distribution of returns with better `mean` and same `std` as the strategy 1*

In [17]:
EXTRA_EDGE = 0.0012

In [18]:
_seed2 = 9000
_sr2  = _sr
_psr2 = _psr

while (_sr2 >= _sr-0.01) or (_psr2 <= 0.95):
    _seed2 += 1
    
    _dist_normal = norm(loc=true_mean+EXTRA_EDGE, scale=true_std)
    _returns2 = _dist_normal.rvs(size=SIZE, random_state=_seed2)
    
    _sr2  = estimated_sharpe_ratio(_returns2)
    _psr2 = probabilistic_sharpe_ratio(_returns2, sr_benchmark=SR_BENCHMARK)

print(_seed2)

9563


In [19]:
print(_sr, _sr2)

0.2262573932753415 0.2150862116259185


In [20]:
print(_psr, _psr2)

0.929881161824999 0.9519351296752798


Save to .csv

In [20]:
pd.DataFrame({'strategy1':_returns, 'strategy2': _returns2}).to_csv('synthetic_returns.csv')